In [1]:
import pandas as pd
import numpy as np
from transformers import AutoModel, AutoTokenizer, AutoConfig
import torch.nn as nn
import torch
from transformers.optimization import get_cosine_schedule_with_warmup
from kobert_tokenizer import KoBERTTokenizer
from torch.utils.data import DataLoader, Dataset, Subset
from sklearn.metrics.pairwise import cosine_similarity
from transformers import BertModel, BertPreTrainedModel
from sklearn.model_selection import KFold
from tqdm import tqdm
import random

In [2]:
#params
batch_size = 16
EPOCHS = 5
PATIENT = 3
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

#fix seed
seed = 511
random.seed(seed)
np.random.seed(seed)
torch.manual_seed(seed)
torch.cuda.manual_seed_all(seed)

In [3]:
#Load dataframe
df_train = pd.read_csv('Train_Part1.csv')
df_test = pd.read_csv('Test_Part1.csv')

In [4]:
#define Dataset
class CustomDataset(Dataset) :
    def __init__(self, dataframe, tokenizer, max_length) :
        self.dataframe = dataframe
        self.tokenizer = tokenizer
        self.max_length = max_length

    def __getitem__(self, x) :
        title = self.tokenize(self.dataframe.loc[x].title)
        context = self.tokenize(self.dataframe.loc[x].content)
        label = torch.tensor(self.dataframe.loc[x].label)
        title_input_ids = title.input_ids.squeeze()
        context_input_ids = context.input_ids.squeeze()
        title_attention_mask = title.attention_mask.squeeze()
        context_attention_mask = context.attention_mask.squeeze()

        return title_input_ids, context_input_ids,title_attention_mask, context_attention_mask, label

    def tokenize(self, data) :
        token = self.tokenizer(data, padding = 'max_length', max_length = self.max_length, truncation = True, return_tensors = "pt")
        del token['token_type_ids']
        return token
    
    def __len__(self) :
        return len(self.dataframe)


In [5]:

class Model(BertPreTrainedModel) :
    def __init__(self, output_classes, config) :
        super().__init__(config)
        self.class_size = config.hidden_size * 2 + batch_size
        self.bert = BertModel.from_pretrained("skt/kobert-base-v1", config=config)
        self.dense = nn.Linear(self.class_size, self.class_size//2)
        self.relu = nn.ReLU()
        self.dense2 = nn.Linear(self.class_size//2, self.class_size//4)
        self.output_layer = nn.Linear(self.class_size//4, output_classes)
        self.dropout = nn.Dropout(config.hidden_dropout_prob)
        
    def forward(self, title_input_ids, title_attention_mask , context_input_ids, context_attention_mask) :
        title_encoded = self.bert(input_ids = title_input_ids,
                                 attention_mask = title_attention_mask)
        context_encoded = self.bert(input_ids = context_input_ids,
                                   attention_mask = context_attention_mask)  
        sim = cosine_similarity(title_encoded.last_hidden_state[:,0,:].to('cpu').detach().numpy(), context_encoded.last_hidden_state[:,0,:].to('cpu').detach().numpy())
        data = torch.cat((title_encoded.last_hidden_state[:,0,:], torch.Tensor(sim)), dim = 1).to(device)
        data = torch.cat((data, context_encoded.last_hidden_state[:,0,:]), dim = 1).to(device)
        
        output = self.relu(self.dense(data))
        output = self.dropout(output)
        output = self.relu(self.dense2(output))
        output = self.dropout(output)
        output = self.output_layer(output)
        return output


In [6]:

tokenizer = KoBERTTokenizer.from_pretrained('skt/kobert-base-v1', last_hidden_states=True)
conf =  AutoConfig.from_pretrained("skt/kobert-base-v1")
model = Model(output_classes = 2, config = conf).to(device)

train_dataset = CustomDataset(dataframe = df_train,
                              tokenizer = tokenizer,
                              max_length = 512)
test_dataset = CustomDataset(dataframe = df_test,
                             tokenizer = tokenizer,
                             max_length = 512)
test_dataloader = DataLoader(test_dataset, batch_size = batch_size, num_workers = 4, shuffle = False)

loss_fn = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.AdamW(
    params       = filter(lambda p: p.requires_grad, model.parameters()), 
    lr           = 0.00001, 
    weight_decay = 0.0005
)

warmup_ratio = 0.1
t_total = len(df_train) * EPOCHS * 4
warmup_step = int(t_total * warmup_ratio)
scheduler = get_cosine_schedule_with_warmup(optimizer, num_warmup_steps = warmup_step, num_training_steps = t_total)


The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'XLNetTokenizer'. 
The class this function is called from is 'KoBERTTokenizer'.
/opt/conda/lib/python3.10/site-packages/torch/_utils.py:831: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  return self.fget.__get__(instance, owner)()


In [7]:
fold = KFold(n_splits = 5, shuffle = True, random_state = seed)

In [8]:

def acc_fn(y_pred, y_true):
    accuracy = torch.eq(y_pred, y_true).sum().item()/len(y_pred)
    return accuracy

def train_func(model, dataloader, optim, loss_fn) :
    t_loss, t_acc = 0, 0
    model.train()
    for batch, (title_input_ids, context_input_ids, title_attention_mask, context_attention_mask, label) in enumerate(tqdm(dataloader)) :
        
        title_input_ids, context_input_ids, title_attention_mask, context_attention_mask, label = title_input_ids.to(device), context_input_ids.to(device), title_attention_mask.to(device), context_attention_mask.to(device), label.to(device) 
        
        outputs = model(title_input_ids = title_input_ids, title_attention_mask =title_attention_mask,
                        context_input_ids = context_input_ids, context_attention_mask = context_attention_mask).to(device)
        outputs = torch.softmax(outputs, dim=1).to(device)

        loss = loss_fn(outputs, label)
        acc = acc_fn(outputs.argmax(dim=1).to(device), label)

        optim.zero_grad()
        loss.backward()
        optim.step()

        t_loss += loss.item()
        t_acc += acc

    t_loss /= len(dataloader)
    t_acc /= len(dataloader)
    return t_loss, t_acc

def eval_func(model, dataloader, loss_fn) :
    e_loss, e_acc = 0, 0
    model.eval()
    with torch.no_grad() :
        for batch, (title_input_ids, context_input_ids, title_attention_mask, context_attention_mask, label) in enumerate(tqdm(dataloader)) :
            title_input_ids, context_input_ids, title_attention_mask, context_attention_mask, label = title_input_ids.to(device), context_input_ids.to(device), title_attention_mask.to(device), context_attention_mask.to(device), label.to(device) 
            outputs = model(title_input_ids = title_input_ids, title_attention_mask =title_attention_mask,
                        context_input_ids = context_input_ids, context_attention_mask = context_attention_mask).to(device)
            outputs = torch.softmax(outputs, dim=1).to(device)
            
            loss = loss_fn(outputs, label)
            acc = acc_fn(outputs.argmax(dim=1).to(device), label.to(device))

            e_loss += loss.item()
            e_acc += acc

        e_loss /= len(dataloader)
        e_acc /= len(dataloader)

    return e_loss, e_acc


In [9]:

def train(train_dataset , model, epochs, optim, loss_fn, patient, scheduler) :
    print('<<TRAIN>>')
    torch.cuda.empty_cache()
    tot_tr_loss, tot_tr_acc = [], []
    tot_val_loss, tot_val_acc = [], []
    for i, (train_idx, val_idx) in enumerate(fold.split(train_dataset)) :
        min_val_loss = 2
        n_patience = 0
        train_ds = Subset(train_dataset, train_idx)
        val_ds = Subset(train_dataset, val_idx)

        train_dl = DataLoader(train_ds, batch_size = batch_size, shuffle = True, num_workers = 4)
        val_dl = DataLoader(val_ds, batch_size = batch_size, shuffle = False, num_workers = 4)

        for epoch in range(epochs) :
            
            print(f'==FOLD {i} / EPOCH {epoch}==')
            train_loss, train_acc = train_func(model, train_dl, optim, loss_fn)
            val_loss, val_acc = eval_func(model, val_dl, loss_fn)
            
            print(f'train_loss / acc : {train_loss} / {train_acc}%, val_loss / acc : {val_loss} / {val_acc}%')
            
            if np.round(min_val_loss, 5) > np.round(val_loss, 5) :
                min_val_loss = val_loss
                n_patience = 0
                print(f'Save the best params with val_loss:{val_loss:.4f}, val_acc:{val_acc:.4f}')
                torch.save(model.state_dict(), f'./models/best_model_fold_{i}.pth')
            else :
                n_patience += 1

            scheduler.step(verbose = True)
            
            if n_patience >= patient :
                print('Early Stopping')
                break

            tot_tr_loss.append(train_loss)
            tot_tr_acc.append(train_acc)
            tot_val_loss.append(val_loss)
            tot_val_acc.append(val_acc)
            print(f'<<FOLD {i}>>')
            print(f"\t Train loss {np.mean(tot_tr_loss):.4f} | acc {np.mean(tot_tr_acc):.4f}")
            print(f"\t Valid loss {np.mean(tot_val_loss):.4f} | acc {np.mean(tot_val_acc):.4f}")


def test(test_dataloader, model , loss_fn, acc_fn) :
    print("<<TEST>>")
    test_loss, test_acc = [],[]
    for i in range(5):
        model.load_state_dict(torch.load(f'./models/best_w_{i}.pth'))
        loss, acc = eval_fn(clf, test_dataloader, loss_fn)
        test_loss.append(loss)
        test_acc.append(acc)

    print(f"Average  loss {np.mean(test_loss):.4f} | acc {np.mean(test_acc):.4f}")
    print(f"Variance loss {np.var(test_loss):.4f} | acc {np.var(test_acc):.4f}")


In [10]:
train(train_dataset = train_dataset,
      model = model,
      epochs = EPOCHS,
      optim = optimizer,
      loss_fn = loss_fn,
      patient = PATIENT,
      scheduler = scheduler)

test(test_dataloader = test_dataloader,
     model = model,
     loss_fn = loss_fn)


<<TRAIN>>
==FOLD 0 / EPOCH 0==


  0%|          | 0/14574 [00:00<?, ?it/s]


OutOfMemoryError: CUDA out of memory. Tried to allocate 192.00 MiB. GPU 0 has a total capacty of 11.76 GiB of which 61.06 MiB is free. Process 4992 has 11.68 GiB memory in use. Of the allocated memory 11.52 GiB is allocated by PyTorch, and 38.46 MiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting max_split_size_mb to avoid fragmentation.  See documentation for Memory Management and PYTORCH_CUDA_ALLOC_CONF